In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from sklearn.model_selection import train_test_split
import classifier as cf
import seaborn as sns
sns.set_style("darkgrid")

In [2]:
traindata = pd.read_csv("data/processed_training_data.csv")

In [3]:
def give_score(row):
    if row["booking_bool"] > 0:
        return int(5)
    if row["click_bool"] > 0:
        return int(1)
    return int(0)

In [4]:
traindata["score"] = traindata.apply(give_score, axis=1)
max(traindata["score"])

5

In [21]:
testdata = pd.read_csv("data/processed_testing_data.csv")

In [6]:
print(traindata.columns)

Index(['Unnamed: 0', 'srch_id', 'date_time', 'site_id',
       'visitor_location_country_id', 'prop_country_id', 'prop_id',
       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_log_historical_price', 'position',
       'price_usd', 'promotion_flag', 'srch_destination_id',
       'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count',
       'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool',
       'random_bool', 'click_bool', 'booking_bool', 'score'],
      dtype='object')


In [8]:
attrs = ["prop_starrating","prop_review_score" ,"prop_location_score1", 
         "prop_log_historical_price", "position", "price_usd", "srch_length_of_stay"]
y = traindata["score"]
X = traindata[attrs]
flag = "True"
#Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=101)

In [8]:
#gnb = cf.gaussian_naive_bayes(Xtrain, Xtest, ytrain, ytest, flag)

Gaussian Naive Bayes: cross vali score 0.9553722208485236 , accuracy score 0.9549873172058438


In [9]:
#dt = cf.decision_tree(Xtrain, Xtest, ytrain, ytest, flag)

Decision Tree: cross vali score 0.9394137529795956 , accuracy score 0.9394844596876148


In [ ]:
#knn = cf.k_nearest_neighbours(Xtrain, Xtest, ytrain, ytest, flag)

In [ ]:
#sgd = cf.stochastic_gradient_descent(Xtrain, Xtest, ytrain, ytest, flag)

In [ ]:
#lr = cf.logistic_regression(Xtrain, Xtest, ytrain, ytest, flag)

In [32]:
import xgboost as xgb
y[np.isnan(y)] = 0
X[np.isnan(X)] = 0
xgmat = xgb.DMatrix(X, y)
# setup parameters for xgboost
param = {}
# use logistic regression loss, use raw prediction before logistic transformation
# since we only need the rank
param['objective'] = 'reg:logistic'
# scale weight of positive examples
param['scale_pos_weight'] = sum(y==0)/sum(y==1)
param['eta'] = 0.1
param['max_depth'] = 6
param['eval_metric'] = 'auc'
param['silent'] = 0
param['subsample'] = 0.8


# you can directly throw param in, though we want to watch multiple metrics here
plst = list(param.items())

watchlist = [ (xgmat,'train') ]

num_round = 150
print ('loading data end, start to boost trees')
bst = xgb.train(plst, xgmat, num_round, watchlist ,early_stopping_rounds=5);
# save out model
bst.save_model(nameOfModel+".model")

print ('finish training')

AttributeError: module 'numpy' has no attribute 'isnull'

In [14]:
len(xgb)

867711

In [22]:
testdata["score"] = xgb.tolist()

ValueError: Length of values (867711) does not match length of index (4959183)

In [20]:
testdata = testdata.sort_values(["srch_id", "score"])
df = pd.DataFrame({"srch_id": testdata["srch_id"].values,
                  "prop_id": testdata["prop_id"]})
df.to_csv("xgb_submission.csv", index=False)
print("done")

done
